**ライブラリのインストール**

PyTorch-Transformers、および必要なライブラリのインストールを行います。

In [1]:
!pip install folium==0.2.1
!pip install urllib3==1.25.11
!pip install pytorch-transformers==1.2.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 957.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79794 sha256=e4c10aee07fbe160e0c99fa20f5803588e778ee35616393f214e7a6d625edba7
  Stored in directory: /root/.cache/pip/wheels/00/0c/07/d7792a5444d5bb074361ac27da53cee9d5cce59a07fe9da5dd
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.14.0
    Uninstalling folium-0.14.0:
      Successfully uninstalled folium-0.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.17.6 requires folium>=0.9.1, but you have folium 0.2.1 which is incompatible.
geemap 0.32.0 requires folium>=0.13.0, but you have folium 0.2.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 2.4 MB/s eta 0:00:00
  

**文章の一部の予測**

文章における一部の単語をMASKし、それをBERTのモデルを使って予測します。

In [2]:

import torch
from pytorch_transformers import BertForMaskedLM
from pytorch_transformers import BertTokenizer


text = "[CLS] I played baseball with my friends at school yesterday [SEP]"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
words = tokenizer.tokenize(text)
print(words)


100%|██████████| 231508/231508 [00:00<00:00, 1308952.41B/s]


['[CLS]', 'i', 'played', 'baseball', 'with', 'my', 'friends', 'at', 'school', 'yesterday', '[SEP]']


文章の一部をMASKします。

In [3]:
msk_idx = 3
words[msk_idx] = "[MASK]"  # 単語を[MASK]に置き換える
print(words)

['[CLS]', 'i', 'played', '[MASK]', 'with', 'my', 'friends', 'at', 'school', 'yesterday', '[SEP]']


単語を対応するインデックスに変換します。

In [4]:
word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
word_tensor = torch.tensor([word_ids])  # テンソルに変換
print(word_tensor)


tensor([[ 101, 1045, 2209,  103, 2007, 2026, 2814, 2012, 2082, 7483,  102]])


BERTのモデルを使って予測を行います。

In [5]:
msk_model = BertForMaskedLM.from_pretrained("bert-base-uncased")
msk_model.cuda()  # GPU対応
msk_model.eval()

x = word_tensor.cuda()  # GPU対応
y = msk_model(x)  # 予測
result = y[0]
print(result.size())  # 結果の形状

_, max_ids = torch.topk(result[0][msk_idx], k=5)  # 最も大きい5つの値
result_words = tokenizer.convert_ids_to_tokens(max_ids.tolist())  # インデックスを単語に変換
print(result_words)

100%|██████████| 440473133/440473133 [00:12<00:00, 34906688.90B/s]


torch.Size([1, 11, 30522])
['basketball', 'football', 'soccer', 'baseball', 'tennis']


 **文章が連続しているかどうかの判定**

BERTのモデルを使って、2つの文章が連続しているかどうかの判定を行います。
以下の関数show_continuityでは、2つの文章の連続性を判定し、表示します。

In [11]:
from pytorch_transformers import BertForNextSentencePrediction

def show_continuity(text, seg_ids):
    words = tokenizer.tokenize(text)
    word_ids = tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
    word_tensor = torch.tensor([word_ids])  # テンソルに変換

    seg_tensor = torch.tensor([seg_ids])

    nsp_model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
    nsp_model.cuda()  # GPU対応
    nsp_model.eval()

    x = word_tensor.cuda()  # GPU対応
    s = seg_tensor.cuda()  # GPU対応

    y = nsp_model(x, s)  # 予測
    result = torch.softmax(y[0], dim=1)
    print(result)  # Softmaxで確率に
    print(str(result[0][0].item()*100) + "%の確率で連続しています。")

show_continuity関数に、自然につながる2つの文章を与えます。

In [12]:
text = "[CLS] What is baseball ? [SEP] It is a game of hitting the ball with the bat [SEP]"
seg_ids = [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1, 1]  # 0:前の文章の単語、1:後の文章の単語
show_continuity(text, seg_ids)

tensor([[1.0000e+00, 4.5869e-06]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
99.9995470046997%の確率で連続しています。


show_continuity関数に、自然につながらない2つの文章を与えます。

In [13]:
text="[CLS] What is baseball ? [SEP] This food is made with flour and milk [SEP]"
seg_ids=[0,0,0,0,0,0,1,1,1,1,1,1,1,1,1]
show_continuity(text,seg_ids)

tensor([[9.5296e-06, 9.9999e-01]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
0.0009529620001558214%の確率で連続しています。
